In [1]:
import pandas as pd
import numpy as np

In [2]:
crispr_df = pd.read_parquet('input/crispr_feat_pert_pval_mwu.parquet').set_index('Feature')
crispr_df.head()

,JCP2022_804842,JCP2022_807651,JCP2022_806473,JCP2022_801888,JCP2022_802452,JCP2022_800101,JCP2022_802960,JCP2022_804292,JCP2022_803465,JCP2022_801838,...,JCP2022_803978,JCP2022_805346,JCP2022_807064,JCP2022_805772,JCP2022_807112,JCP2022_806737,JCP2022_806893,JCP2022_803210,JCP2022_807690,JCP2022_805881
Feature,,,,,,,,,,,,,,,,,,,,,
Cells_AreaShape_Area,0.000666,0.000666,0.234324,0.000666,0.007992,0.000666,0.439560,0.000666,0.027972,0.000103,...,0.000666,0.229535,0.161251,0.000103,0.075258,0.019314,0.002997,0.043089,0.016929,0.055944
Cells_AreaShape_BoundingBoxArea,0.000666,0.000666,1.000000,0.000666,0.027972,0.000666,0.439560,0.000666,0.012654,0.000103,...,0.000666,0.229535,0.024990,0.000103,0.075258,0.019314,0.002997,0.043089,0.005857,0.072677
Cells_AreaShape_BoundingBoxMaximum_X,0.129204,0.953047,0.798966,0.075258,0.206460,0.513487,0.767899,0.370962,0.055278,0.070239,...,0.039960,0.024990,0.087824,0.314788,0.859141,0.206460,0.117882,0.600884,0.668170,0.093407
Cells_AreaShape_BoundingBoxMaximum_Y,0.254412,0.164502,0.234324,0.254412,0.019314,0.370962,0.075258,0.004662,0.019314,0.009667,...,0.594073,0.669066,0.000206,0.812526,0.594073,0.002664,0.001748,0.363842,0.425983,0.313187
Cells_AreaShape_BoundingBoxMinimum_X,0.129204,0.767899,0.878798,0.027972,0.075258,0.594073,0.370962,0.678655,0.027972,0.018511,...,0.007992,0.033011,0.043089,0.108803,0.513487,0.075258,0.007493,0.536199,0.500559,0.072677


In [3]:
crispr_df = (
    crispr_df.stack()
    .reset_index()
    .rename(columns={"level_1": "Metadata_JCP2022", 0: "pval"})
    .query("pval < 0.05")
)

crispr_df = crispr_df.groupby("Metadata_JCP2022").Feature.apply(list).reset_index()

crispr_df.head()

,Metadata_JCP2022,Feature
0,JCP2022_800003,"[Cells_AreaShape_Area, Cells_AreaShape_Boundin..."
1,JCP2022_800004,"[Cells_AreaShape_Area, Cells_AreaShape_Boundin..."
2,JCP2022_800005,"[Cells_AreaShape_BoundingBoxMaximum_Y, Cells_A..."
3,JCP2022_800006,"[Cells_AreaShape_Area, Cells_AreaShape_Boundin..."
4,JCP2022_800007,"[Cells_AreaShape_Area, Cells_AreaShape_Boundin..."


In [4]:
crispr_metadata_df = pd.read_csv(
    "../00.download-and-process-annotations/output/crispr_metadata.tsv.gz", sep="\t"
)[
    [
        "Metadata_JCP2022",
        "Metadata_Approved_Symbol",
        "Metadata_biological_process",
        "Metadata_cellular_component",
    ]
]
crispr_metadata_df.head()

,Metadata_JCP2022,Metadata_Approved_Symbol,Metadata_biological_process,Metadata_cellular_component
0,JCP2022_800001,NaN,NaN,NaN
1,JCP2022_800002,NaN,NaN,NaN
2,JCP2022_800003,A2M,GOBP_ACTIVATION_OF_IMMUNE_RESPONSE|GOBP_ACUTE_...,GOCC_BLOOD_MICROPARTICLE|GOCC_COLLAGEN_CONTAIN...
3,JCP2022_800004,A3GALT2,GOBP_CARBOHYDRATE_DERIVATIVE_BIOSYNTHETIC_PROC...,GOCC_GOLGI_APPARATUS|GOCC_GOLGI_APPARATUS_SUBC...
4,JCP2022_800005,A4GALT,GOBP_CARBOHYDRATE_DERIVATIVE_BIOSYNTHETIC_PROC...,GOCC_GOLGI_APPARATUS|GOCC_GOLGI_MEMBRANE


In [5]:
metadata_df = pd.concat(
    [
        crispr_metadata_df.dropna(subset=["Metadata_Approved_Symbol"])[
            crispr_metadata_df.dropna(
                subset=["Metadata_Approved_Symbol"]
            ).Metadata_Approved_Symbol.str.startswith("ATP6V")
        ].assign(complex="Vacuolar-type ATPase"),
        crispr_metadata_df.dropna(subset=["Metadata_biological_process"])[
            crispr_metadata_df.dropna(
                subset=["Metadata_biological_process"]
            ).Metadata_biological_process.str.contains(
                "GOBP_PROTEIN_O_LINKED_MANNOSYLATION"
            )
        ].assign(complex="Protein O-mannosylation"),
        crispr_metadata_df.dropna(subset=["Metadata_biological_process"])[
            crispr_metadata_df.dropna(
                subset=["Metadata_biological_process"]
            ).Metadata_biological_process.str.contains(
                "GOBP_OUTER_MITOCHONDRIAL_MEMBRANE_ORGANIZATION"
            )
        ].assign(complex="OMM Protein Complex"),
        crispr_metadata_df.dropna(subset=["Metadata_biological_process"])[
            crispr_metadata_df.dropna(
                subset=["Metadata_biological_process"]
            ).Metadata_biological_process.str.contains(
                "GOBP_CORTICAL_CYTOSKELETON_ORGANIZATION"
            )
        ].assign(complex="Cortical Cytoskeleton"),
        crispr_metadata_df.dropna(subset=["Metadata_cellular_component"])[
            crispr_metadata_df.dropna(
                subset=["Metadata_cellular_component"]
            ).Metadata_cellular_component.str.contains("GOCC_DNA_POLYMERASE_COMPLEX")
        ].assign(complex="DNA Polymerase Complex"),
    ],
    ignore_index=True,
)

In [6]:
crispr_df = crispr_df.merge(metadata_df, on="Metadata_JCP2022", how="inner")
crispr_df.head()

,Metadata_JCP2022,Feature,Metadata_Approved_Symbol,Metadata_biological_process,Metadata_cellular_component,complex
0,JCP2022_800349,"[Cells_AreaShape_Area, Cells_AreaShape_Boundin...",AKAP11,GOBP_ACTIN_FILAMENT_BASED_PROCESS|GOBP_CHEMICA...,GOCC_MICROTUBULE_CYTOSKELETON|GOCC_MICROTUBULE...,Cortical Cytoskeleton
1,JCP2022_800556,"[Cells_AreaShape_Area, Cells_AreaShape_Boundin...",ARF6,GOBP_ACTIN_FILAMENT_BASED_PROCESS|GOBP_ACTIN_F...,GOCC_ACTIN_BASED_CELL_PROJECTION|GOCC_ANCHORIN...,Cortical Cytoskeleton
2,JCP2022_800729,"[Cells_AreaShape_Area, Cells_AreaShape_Boundin...",ATP6V0A4,GOBP_CELLULAR_HOMEOSTASIS|GOBP_CHEMICAL_HOMEOS...,GOCC_APICAL_PART_OF_CELL|GOCC_ATPASE_COMPLEX|G...,Vacuolar-type ATPase
3,JCP2022_800730,"[Cells_AreaShape_Area, Cells_AreaShape_Boundin...",ATP6V0D1,GOBP_CELLULAR_HOMEOSTASIS|GOBP_CELLULAR_RESPON...,GOCC_APICAL_PART_OF_CELL|GOCC_ATPASE_COMPLEX|G...,Vacuolar-type ATPase
4,JCP2022_800731,"[Cells_AreaShape_Area, Cells_AreaShape_Boundin...",ATP6V0D2,GOBP_CELLULAR_HOMEOSTASIS|GOBP_CHEMICAL_HOMEOS...,GOCC_APICAL_PART_OF_CELL|GOCC_ATPASE_COMPLEX|G...,Vacuolar-type ATPase


In [7]:
Channels = ["ER", "Mito", "DNA", "AGP", "RNA"]

for channel in Channels:
    crispr_df = crispr_df.assign(
        count=lambda x: x.Feature.apply(
            lambda y: sum([1 if channel in i else 0 for i in y]) 
        )
    ).rename(columns={"count": f"{channel}_count"})

crispr_df.head()

,Metadata_JCP2022,Feature,Metadata_Approved_Symbol,Metadata_biological_process,Metadata_cellular_component,complex,ER_count,Mito_count,DNA_count,AGP_count,RNA_count
0,JCP2022_800349,"[Cells_AreaShape_Area, Cells_AreaShape_Boundin...",AKAP11,GOBP_ACTIN_FILAMENT_BASED_PROCESS|GOBP_CHEMICA...,GOCC_MICROTUBULE_CYTOSKELETON|GOCC_MICROTUBULE...,Cortical Cytoskeleton,433,479,343,345,456
1,JCP2022_800556,"[Cells_AreaShape_Area, Cells_AreaShape_Boundin...",ARF6,GOBP_ACTIN_FILAMENT_BASED_PROCESS|GOBP_ACTIN_F...,GOCC_ACTIN_BASED_CELL_PROJECTION|GOCC_ANCHORIN...,Cortical Cytoskeleton,347,324,103,299,338
2,JCP2022_800729,"[Cells_AreaShape_Area, Cells_AreaShape_Boundin...",ATP6V0A4,GOBP_CELLULAR_HOMEOSTASIS|GOBP_CHEMICAL_HOMEOS...,GOCC_APICAL_PART_OF_CELL|GOCC_ATPASE_COMPLEX|G...,Vacuolar-type ATPase,319,453,169,267,372
3,JCP2022_800730,"[Cells_AreaShape_Area, Cells_AreaShape_Boundin...",ATP6V0D1,GOBP_CELLULAR_HOMEOSTASIS|GOBP_CELLULAR_RESPON...,GOCC_APICAL_PART_OF_CELL|GOCC_ATPASE_COMPLEX|G...,Vacuolar-type ATPase,414,347,527,297,434
4,JCP2022_800731,"[Cells_AreaShape_Area, Cells_AreaShape_Boundin...",ATP6V0D2,GOBP_CELLULAR_HOMEOSTASIS|GOBP_CHEMICAL_HOMEOS...,GOCC_APICAL_PART_OF_CELL|GOCC_ATPASE_COMPLEX|G...,Vacuolar-type ATPase,256,392,121,246,274


In [8]:
crispr_feature_count = (
    crispr_df[
        ["complex", "ER_count", "Mito_count", "DNA_count", "AGP_count", "RNA_count"]
    ]
    .groupby("complex")
    .apply(lambda x: x.mean(numeric_only=True))
)

crispr_feature_count.head()

,ER_count,Mito_count,DNA_count,AGP_count,RNA_count
complex,,,,,
Cortical Cytoskeleton,337.166667,368.611111,195.277778,276.944444,341.444444
DNA Polymerase Complex,286.764706,345.941176,160.176471,254.352941,264.000000
OMM Protein Complex,257.666667,320.666667,206.333333,217.666667,288.333333
Protein O-mannosylation,285.333333,296.833333,158.166667,268.166667,277.500000
Vacuolar-type ATPase,321.187500,353.875000,182.562500,291.187500,330.750000


In [9]:
cripsr_feature_percent = crispr_feature_count.div(crispr_feature_count.sum(axis=1), axis=0)

cripsr_feature_percent

,ER_count,Mito_count,DNA_count,AGP_count,RNA_count
complex,,,,,
Cortical Cytoskeleton,0.221901,0.242596,0.128519,0.182267,0.224717
DNA Polymerase Complex,0.218698,0.263828,0.122157,0.193980,0.201337
OMM Protein Complex,0.199638,0.248450,0.159866,0.168647,0.223399
Protein O-mannosylation,0.221877,0.230819,0.122991,0.208528,0.215785
Vacuolar-type ATPase,0.217083,0.239175,0.123390,0.196806,0.223546


In [10]:
print(cripsr_feature_percent.to_markdown())

| complex                 |   ER_count |   Mito_count |   DNA_count |   AGP_count |   RNA_count |
|:------------------------|-----------:|-------------:|------------:|------------:|------------:|
| Cortical Cytoskeleton   |   0.221901 |     0.242596 |    0.128519 |    0.182267 |    0.224717 |
| DNA Polymerase Complex  |   0.218698 |     0.263828 |    0.122157 |    0.19398  |    0.201337 |
| OMM Protein Complex     |   0.199638 |     0.24845  |    0.159866 |    0.168647 |    0.223399 |
| Protein O-mannosylation |   0.221877 |     0.230819 |    0.122991 |    0.208528 |    0.215785 |
| Vacuolar-type ATPase    |   0.217083 |     0.239175 |    0.12339  |    0.196806 |    0.223546 |
